# Retriever
- 비정형 질의(query)를 입력 받아 Vector store에서 관련된 내용을 검색하는 기능을 제공하는 인터페이스로 다양한 데이터 소스에서 정보를 검색하여 대규모 언어 모델(LLM) 기반 애플리케이션의 **정확성을** 향상시키는 데 핵심적인 역할을 한다.

![RAG](figures/rag2.png)

## 주요 특징
- **다양한 데이터 소스 지원**
	- Retriever는 벡터 스토어, 그래프 데이터베이스, 관계형 데이터베이스 등 여러 종류의 검색 시스템과 상호작용할 수 있는 통일된 인터페이스를 제공한다다.
- **간단한 인터페이스**: Retriever는 문자열 형태의 쿼리를 입력받아 관련 문서의 리스트를 반환한다. 이러한 단순한 구조 덕분에 다양한 검색 시스템과 쉽게 통합할 수 있다. 


## 다양한 Retriever 방식

**Retriever**란, 사용자의 질문(쿼리)에 가장 관련성 높은 정보를 찾아주는 구성 요소이다. 주로 검색 기반 질문응답 시스템(RAG, Retrieval-Augmented Generation)에서 사용된다. 다음은 자주 사용되는 다양한 Retriever의 유형과 그 특징이다.

1. **벡터 스토어(Vector Store) Retriever**
   - VectorStore로 부터 유사도를 기반으로 검색하는 가장 기본 Retriever
   - 텍스트 조각(청크)마다 **임베딩(embedding)을** 생성하여 벡터 공간에 저장하고, 쿼리 임베딩과의 **코사인 유사도(cosine similarity)** 등을 기반으로 유사한 텍스트를 검색한다.
   - 검색 속도가 빠르고 구현이 간단하여, 기본적인 검색 시스템을 구축할 때 적합하다.
2. **[ParentDocumentRetriever](https://python.langchain.com/docs/how_to/parent_document_retriever/)**
   - 하나의 문서를 여러 청크로 나눈 뒤 각각을 인덱싱하고, 쿼리와 가장 유사한 청크를 찾은 다음 해당 청크가 속한 **전체 원본 문서**를 반환한다.
   - 작은 정보 조각들이 모여 하나의 문서를 구성할 때 유용하며, 문맥을 넓게 유지할 수 있다.
3. **[MultiVectorRetriever](https://python.langchain.com/docs/how_to/multi_vector/)**
   - 각 문서에 대해 요약을 하거나, 가상의 질문을 생성하거나, 사람이 중요한 내용을 직접 추가하여 문서당 여러 개의 임베딩 벡터를 생성한다.
   - 텍스트 전체보다 더 핵심적인 정보가 검색에 반영되도록 하고자 할 때 효과적이다.
   - 특히, 문서가 길거나, 중요한 내용이 문서의 특정 부분에 집중되어 있는 경우에 유리하다.
4. **[SelfQueryRetriever](https://python.langchain.com/docs/how_to/self_query/)**
   - 대규모 언어 모델(LLM, Large Language Model)을 활용하여 사용자의 질문을 적절한 검색어와 **메타데이터(metadata)** 필터로 자동 변환한다.
   - 예를 들어, 문서의 작성자, 날짜, 태그와 같은 메타데이터를 기준으로 검색할 수 있다.
   - 문서 자체의 내용뿐만 아니라, 문서에 부가된 속성 정보에 대해 질문할 때 유용하다.
5. **[ContextualCompressionRetriever](https://python.langchain.com/docs/how_to/contextual_compression/)**
   - 기존 Retriever와 조합되어 사용된다.
   - 먼저 일반적인 검색을 수행한 후, 검색된 문서들에서 쿼리와 관련 없는 불필요한 정보를 제거하고 핵심 내용만을 추출하여 반환한다.
   - 정보를 요약하거나 압축하여 LLM에 전달할 문서 길이를 줄일 때 유용하다.
6. **[MultiQueryRetriever](https://python.langchain.com/docs/how_to/MultiQueryRetriever/)**
   - LLM을 이용해 하나의 쿼리로부터 여러 가지 변형된 쿼리를 생성하고, 각 쿼리에 대해 검색을 수행한 뒤 결과를 합치는 방식.
   - 다양한 표현에 강해 검색 범위를 넓히고 성능을 높인다.
7. **[EnsembleRetriever](https://python.langchain.com/docs/how_to/ensemble_retriever/)**
   - 여러 개의 Retriever(예: BM25, 벡터 기반 등)를 결합해 가중치를 기반으로 결과를 조합(re-ranking)한다.
   - 서로 다른 장점을 가진 Retriever를 하나로 묶어 성능을 강화한다.

# TODO 다음을 작성한다.

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import MarkdownHeaderTextSplitter

from langchain_qdrant import QdrantVectorStore, FastEmbedSparse, RetrievalMode
from qdrant_client import QdrantClient
from qdrant_client.models import Distance, SparseVectorParams, VectorParams, SparseIndexParams


loader = TextLoader("data/olympic_wiki.md", encoding="utf-8")

headers_to_split_on = [
    ("#", "H1"),
    ("##", "H2"),
    ("###", "H3"),
]

splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on = headers_to_split_on
)

documents = loader.load()  # list[Document]
# 리스트의 document들의 내용을 모두 추출해서 하나의 문자열로 만든 뒤 split
docs = splitter.split_text('\n\n'.join([doc.page_content for doc in documents]))
print(len(docs))

25


In [3]:
COLLECTION_NAME = "olympic_info"
VECTOR_SIZE = 3072  # OpenAI text-embedding-3-large 벡터 차원

dense_embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
sparse_embeddings = FastEmbedSparse(model_name="Qdrant/bm25")

####### Qdrant Client 생성(연결)
client = QdrantClient(host="localhost", port=6333)

######### Collection 삭제
if client.collection_exists(collection_name=COLLECTION_NAME):
    result = client.delete_collection(collection_name=COLLECTION_NAME)
    print("삭제:", result)


####################### Collection 생성
## sparse, dencse 설정 모두 
client.create_collection(
    collection_name=COLLECTION_NAME,
    vectors_config={
        "dense": VectorParams(  # dense: 벡터 인덱스의 이름.
            size=VECTOR_SIZE,
            distance=Distance.COSINE
        )
    },
    sparse_vectors_config={ 
        "sparse": SparseVectorParams(index=SparseIndexParams(on_disk=False)) # 이 설정을 통해 sparse vector도 인덱싱 되도록 한다. (설정은 SparseIndexParams로한다. on_disk=False: 메모리 인덱스 하겠다 설정(이것때문에 SparseIndexParams를 사용한 것은 아님))
    }
)

삭제: True


True

In [4]:
vector_store = QdrantVectorStore(
    client=client,
    collection_name=COLLECTION_NAME,
   
    embedding=dense_embeddings,                  # Dense Embedding 모델
    sparse_embedding=sparse_embeddings,    # Sparse Embedding 모델
   
    retrieval_mode=RetrievalMode.HYBRID,  # 모드를 HYBRID로 설정
   
    vector_name="dense",                   # 벡터 공간의 이름(collection 생성시 지정한 이름으로 해야 한다.)
    sparse_vector_name="sparse",           # Sparse 벡터 공간의 이름(collection 생성시 지정한 이름으로 해야 한다.)
)

#################### load documents vector store에 저장
vector_store.add_documents(documents=docs)

['6a17ba58b95b4d628c057e5bcaf2b108',
 '11423bdfc7764328b602f4d71eace0f1',
 'a42d966a20ee499d97b827778faa32bd',
 '9f6123d81c8344239e5031ed92637e07',
 '29e3ec3307de4eab802cdd4ea3868dbb',
 '991655bce5d248c9a936a0ee75a8bcc3',
 'd36715a95b7347d09e921ea874932f43',
 'd470499cc8404769828ec6fe03cc305a',
 '5c7c612806554f009e61587b9e3db87f',
 '05619a976f0b499cbae8f88e863a4860',
 'aa543893b20c4ad7877b2a1c43e302a3',
 'ce7ae5853d9f46c3a3ee7601c99cf383',
 '3f3ef34ccd4f4c36881d48fb0b29027b',
 'b1a7c45fe2f84d9fa2aa0ca8ccdd7592',
 '4467f6785c974ebeaf6d0d5aae1759d9',
 '764a235cf1ff4e3081823e83cca20354',
 'c6f733f77cc848799ea14ae62d0f14af',
 'b022ab7d43ee4cc6862bba1d69d603b8',
 '741e17c6c4be4b359f156b800d66f670',
 'ad6b80dfdafd479ba064dfa07f53a052',
 '0c8a63fe239d401e9b4d64d42d225717',
 '917639c9af414af1900ee6458f1a7d94',
 'ce49083d5ed04638b87fff0d2259f229',
 '4801a39410424fa8a1b1ebb3f40f2337',
 'c1e19b6f0e3241f49ee8d3b7e1d4a851']

In [6]:
# Retriever 생성
##  vectorstore.as_retriever(검색관련 설정)
## 검색관련설정: similarity_search()에서 넣어준 파라미터들 (query빼고.)

retriever = vector_store.as_retriever() # default: k=4

# Retriever를 이용해서 검색. Retriever: Runnable  타입
# invoke(query:str)
result = retriever.invoke("1회 동계올림픽은 언제 어디에서 개최했지?")
print(len(result))
result

4


[Document(metadata={'H1': '올림픽', 'H2': '근대 올림픽', 'H3': '동계 올림픽', '_id': '29e3ec33-07de-4eab-802c-dd4ea3868dbb', '_collection_name': 'olympic_info'}, page_content='동계 올림픽은 눈과 얼음을 이용하는 스포츠들을 모아 이루어졌으며 하계 올림픽 때 실행하기 불가능한 종목들로 구성되어 있다. 피겨스케이팅, 아이스하키는 각각 1908년과 1920년에 하계올림픽 종목으로 들어가 있었다. IOC는 다른 동계 스포츠로 구성된 새로운 대회를 만들고 싶어 했고, 로잔에서 열린 1921년 올림픽 의회에서 겨울판 올림픽을 열기로 합의했다. 1회 동계올림픽은 1924년, 프랑스의 샤모니에서 11일간 진행되었고, 16개 종목의 경기가 치러졌다. IOC는 동계 올림픽이 4년 주기로 하계 올림픽과 같은 년도에 열리도록 했다. 이 전통은 프랑스의 알베르빌에서 열린 1992년 올림픽 때까지 지속되었으나, 노르웨이의 릴레함메르에서 열린 1994년 올림픽부터 동계 올림픽은 하계 올림픽이 끝난지 2년후에 개최하였다.'),
 Document(metadata={'H1': '올림픽', 'H2': '개최지 선정', '_id': 'c1e19b6f-0e32-41f4-9ee8-d3b7e1d4a851', '_collection_name': 'olympic_info'}, page_content='올림픽 개최지는 해당 올림픽 개최 7년 전에 IOC 위원들의 투표로 결정된다. 개최지 선정에는 약 2년이 걸린다. 유치를 희망하는 도시는 우선 자국의 올림픽 위원회에 신청을 해야 한다. 만약 한 국가에서 두 도시 이상이 유치를 희망한다면, 한 국가당 한 도시만 후보가 될 수 있다는 규칙에 따라 내부적으로 후보 도시를 결정해야 한다. 후보 도시가 결정되면 후보 도시가 소속된 국가의 올림픽 위원회는 IOC에 개최 신청을 하고, 신청 후에는 올림픽 개최에 대한 질의 응답서를 보내야 한다. 이 질의응답

In [9]:
# search_type: 검색 방식
## "similarity"(default), "similarity_score_threshold": score_threshold를 지정가능., "mmr" - 다양성/관련성 조화를 조정
# 추가 설정
## search_kwargs: dict로 설정.
retriever2 = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={
        "k":7,
        "score_threshold":0.7
    }
)
result = retriever2.invoke("올림픽과 관련된 논란들은 어떤 것들이 있었나?")
print(len(result))
result

3


[Document(metadata={'H1': '올림픽', 'H2': '논란', 'H3': '정치', '_id': 'ad6b80df-dafd-479b-a064-dfa07f53a052', '_collection_name': 'olympic_info'}, page_content="쿠베르탱이 말했던 원래 이념과는 반대로 올림픽이 정치 혹은 체제 선전의 장으로 이용되는 경우가 있었다. 1936년 하계 올림픽을 개최할 때 당시의 나치독일은 나치는 자비롭고 평화를 위한다는 것을 설명하고 싶어했다. 또 이 올림픽에서 아리안족의 우월함을 보여줄 생각이었으나 이는 흑인이었던 제시 오언스가 금메달을 4개나 따내면서 실현되지는 못했다. 소련은 헬싱키에서 열린 1952년 하계 올림픽 때 처음으로 참가했다. 그 전에는 소련이 조직한 스파르타키아다라는 대회에 1928년부터 참가했었다. 다른 공산주의 국가들은 1920년대와 1930년대의 전쟁 기간 사이에 노동자 올림픽(Socialist Workers' Sport International)을 조직했는데, 이는 올림픽을 자본가와 귀족들의 대회로 여기고 그에 대한 대안으로 고안된 대회였다. 그 이후 소련은 1956년 하계 올림픽부터 1988년 하계 올림픽까지 엄청난 스포츠강국의 면모를 보여주며 올림픽에서의 명성을 드높였다.  \n선수 개인이 자신의 정치적 성향에 대해 표현하기도 했다. 멕시코 시티에서 열린 1968년 하계 올림픽의 육상부문 200m 경기에서 각각 1위와 3위를 한 미국의 토미 스미스와 존 카를로스는 시상식 때 블랙 파워 설루트(Black Power salute , 흑인 차별 반대 행위)를 선보였으며 2위를 한 피터 노먼도 상황을 깨닫고 스미스와 카를로스의 행위를 지지한다는 뜻에서 급하게 인권을 위한 올림픽 프로젝트(OPHR) 배지를 달았다. 이 사건에 대해서 IOC 위원장이었던 에이버리 브런디지는 미국 올림픽 위원회에 이 두 선수를 미국으로 돌려보내거나 미국 육상팀 전부를 돌려보내는 둘 중 하나의 선택을 하게 했고, 미국 올

In [11]:
# mmr 타입(방식)
retriever3 = vector_store.as_retriever(
    search_type='mmr',
    search_kwargs={
        "k":5,
        "fetch_k": 10,
        "lambda_mult":0.5 
    }
)
result = retriever3.invoke("올림픽과 관련된 논란들은 어떤 것들이 있었나?")
print(len(result))
result

5


[Document(metadata={'H1': '올림픽', 'H2': '논란', 'H3': '정치', '_id': 'ad6b80df-dafd-479b-a064-dfa07f53a052', '_collection_name': 'olympic_info'}, page_content="쿠베르탱이 말했던 원래 이념과는 반대로 올림픽이 정치 혹은 체제 선전의 장으로 이용되는 경우가 있었다. 1936년 하계 올림픽을 개최할 때 당시의 나치독일은 나치는 자비롭고 평화를 위한다는 것을 설명하고 싶어했다. 또 이 올림픽에서 아리안족의 우월함을 보여줄 생각이었으나 이는 흑인이었던 제시 오언스가 금메달을 4개나 따내면서 실현되지는 못했다. 소련은 헬싱키에서 열린 1952년 하계 올림픽 때 처음으로 참가했다. 그 전에는 소련이 조직한 스파르타키아다라는 대회에 1928년부터 참가했었다. 다른 공산주의 국가들은 1920년대와 1930년대의 전쟁 기간 사이에 노동자 올림픽(Socialist Workers' Sport International)을 조직했는데, 이는 올림픽을 자본가와 귀족들의 대회로 여기고 그에 대한 대안으로 고안된 대회였다. 그 이후 소련은 1956년 하계 올림픽부터 1988년 하계 올림픽까지 엄청난 스포츠강국의 면모를 보여주며 올림픽에서의 명성을 드높였다.  \n선수 개인이 자신의 정치적 성향에 대해 표현하기도 했다. 멕시코 시티에서 열린 1968년 하계 올림픽의 육상부문 200m 경기에서 각각 1위와 3위를 한 미국의 토미 스미스와 존 카를로스는 시상식 때 블랙 파워 설루트(Black Power salute , 흑인 차별 반대 행위)를 선보였으며 2위를 한 피터 노먼도 상황을 깨닫고 스미스와 카를로스의 행위를 지지한다는 뜻에서 급하게 인권을 위한 올림픽 프로젝트(OPHR) 배지를 달았다. 이 사건에 대해서 IOC 위원장이었던 에이버리 브런디지는 미국 올림픽 위원회에 이 두 선수를 미국으로 돌려보내거나 미국 육상팀 전부를 돌려보내는 둘 중 하나의 선택을 하게 했고, 미국 올

In [18]:
##############################
# payload(metadata) filtering 
##############################
from qdrant_client.models import Filter, FieldCondition, MatchValue
filter_condition = Filter(
    must=[
        FieldCondition(key="metadata.H2", match=MatchValue(value="논란"))
    ]
)

retriever4 = vector_store.as_retriever(
    search_kwargs={
        "k":5,
        "filter":filter_condition
    }
)
# result = retriever4.invoke("올림픽과 관련된 논란들은 어떤 것들이 있었나?")
result = retriever4.invoke("동계 올림픽 종목에는 무엇이있나요?") 
# filter가 고정되어 H2가 논란인 것 안에서 조회

print(len(result))
result

4


[Document(metadata={'H1': '올림픽', 'H2': '논란', 'H3': '폭력 및 전쟁', '_id': '917639c9-af41-4af1-900e-e6458f1a7d94', '_collection_name': 'olympic_info'}, page_content='쿠베르탱의 생각과는 달리, 올림픽이 세계에 완벽한 평화를 가져다주지는 못했다. 실제로 제1차 세계대전으로 인해 독일 베를린에서 열리기로 했던 제6회 1916년 하계 올림픽이 취소되었고, 제2차 세계대전 때는 일본 도쿄에서 열리기로 했던 제12회 1940년 하계 올림픽, 삿포로에서 열리기로 했던 1940년 동계 올림픽, 영국 런던에서 열리기로 했던 제13회 1944년 하계 올림픽, 이탈리아 코르티나담페초에서 열릴 예정인 1944년 동계 올림픽이 취소되었다. 베이징에서 열린 2008년 하계 올림픽 개막식날 조지아와 러시아 간의 2008년 남오세티아 전쟁이 일어나기도 했다. 부시 대통령과 푸틴 대통령이 이 올림픽을 보러 왔으며 중국 주석인 후진타오가 주최한 오찬에 참석해서 이 현안에 대해 논의하기도 했다. 조지아 대표인 니노 살루크바체와 러시아 대표인 나탈리야 파데리나가 여자 10m 공기권총 경기에서 각각 동메달과 은메달을 땄을 때 이 일은 베이징 올림픽의 유명한 사건 중 하나로 남게 되었다. 살루크바체와 파데리나는 시상식이 끝난 뒤 서로 포옹을 하며 국적에 상관없이 기쁨을 나누었다.  \n테러도 올림픽에서 공포의 대상이었다. 뮌헨 참사로 알려진 1972년에 서독 바이에른의 뮌헨에서 열린 하계 올림픽때의 사건은 테러리스트인 검은 9월단이 일으킨 사건으로서 이스라엘 선수 11명을 인질로 붙잡았다가 전원이 사망한 사건이다. 당시 미숙한 진압으로 인해 인질 9명(선수 1명과 코치 1명은 인질로 잡기 이전에 살해), 테러범 5명, 독일 경찰관 1명이 사망했으며 이 진압 작전 이전에는 인질들은 단 한 명도 죽지 않았다. 애틀랜타에서 열린 1996년 하계 올림픽 때는 센테니얼 올림픽 공원(Cen

In [19]:
##############################################
#  Retriever의 설정들 invoke() 시 동적으로 변경
##############################################
# retriever4의 어떤 설정을 바꿀 수있는지 지정해서 retriever를 생성.
from langchain_core.runnables import ConfigurableField
retriever5 = retriever4.configurable_fields(
    # 바꿀 파라미터 = ConfigurableField()
    search_type=ConfigurableField(
        id="search_type", # invoke() 바꿀 때 지정할 key
    ),
    search_kwargs=ConfigurableField(
        id="search_kwargs"
    )
)
# retriever5 는 retriever4에서 search_type과 search_kwargs를 invoke() 시 변경할 수있는
# retriever이다.

# invoke(query, config: RunnableConfig)  # config에서 설정을 변경.

In [20]:
# 일부 설정만 변경할 수는 없다.
# search_kwargs: {k:5, filter=f_c} ===> {k:10}
config = {
    "configurable":{
        "search_kwargs":{
            "k":10
        }
    }
}
query = "동계 올림픽 종목에는 무엇이있나요?"
retriever5.invoke(query, config=config)

[Document(metadata={'H1': '올림픽', 'H2': '근대 올림픽', 'H3': '동계 올림픽', '_id': '29e3ec33-07de-4eab-802c-dd4ea3868dbb', '_collection_name': 'olympic_info'}, page_content='동계 올림픽은 눈과 얼음을 이용하는 스포츠들을 모아 이루어졌으며 하계 올림픽 때 실행하기 불가능한 종목들로 구성되어 있다. 피겨스케이팅, 아이스하키는 각각 1908년과 1920년에 하계올림픽 종목으로 들어가 있었다. IOC는 다른 동계 스포츠로 구성된 새로운 대회를 만들고 싶어 했고, 로잔에서 열린 1921년 올림픽 의회에서 겨울판 올림픽을 열기로 합의했다. 1회 동계올림픽은 1924년, 프랑스의 샤모니에서 11일간 진행되었고, 16개 종목의 경기가 치러졌다. IOC는 동계 올림픽이 4년 주기로 하계 올림픽과 같은 년도에 열리도록 했다. 이 전통은 프랑스의 알베르빌에서 열린 1992년 올림픽 때까지 지속되었으나, 노르웨이의 릴레함메르에서 열린 1994년 올림픽부터 동계 올림픽은 하계 올림픽이 끝난지 2년후에 개최하였다.'),
 Document(metadata={'H1': '올림픽', '_id': '6a17ba58-b95b-4d62-8c05-7e5bcaf2b108', '_collection_name': 'olympic_info'}, page_content='올림픽(영어: Olympic Games, 프랑스어: Jeux olympiques)은 전 세계 각 대륙 각국에서 모인 수천 명의 선수가 참가해 여름과 겨울에 스포츠 경기를 하는 국제적인 대회이다. 전 세계에서 가장 큰 지구촌 최대의 스포츠 축제인 올림픽은 세계에서 가장 인지도있는 국제 행사이다. 올림픽은 2년마다 하계 올림픽과 동계 올림픽이 번갈아 열리며, 국제 올림픽 위원회(IOC)가 감독하고 있다. 또한 오늘날의 올림픽은 기원전 8세기부터 서기 5세기에 이르기까지 고대 그리스 올림피아에서 열렸던 올림피아 제전에서 비롯되었

In [23]:
config = {
    "configurable": {
        "search_type":"mmr",
        "search_kwargs": {
            "k":5,
            "fetch_k":10,
            "lambda_mult": 0.2,
            # "filter":filter_condition
        }
    }
}
query = "올림픽에서 약물과 관련된 스캔들이 뭐가 있어지?"
retriever5.invoke(query, config=config)

[Document(metadata={'H1': '올림픽', 'H2': '논란', 'H3': '약물 복용', '_id': '0c8a63fe-239d-401e-9b4d-64d42d225717', '_collection_name': 'olympic_info'}, page_content='20세기 초반, 많은 운동 선수들은 기록향상을 위해 약물을 복용하기 시작했다. 예를 들어 1904년 하계 올림픽 마라톤에서 우승한 미국 선수 토머스 J. 힉스는 코치에게서 스트리크닌과 브랜디를 받았다. 올림픽에서 약물을 과다 복용으로 사망한 사례도 한 번 있었다. 1960년 로마 대회 때 사이클 개인도로 경기 중에 덴마크 선수인 크누드 에네마르크 옌센이 자전거에서 떨어져서 사망했다. 검시관들의 조사에 의하면 그의 죽음의 원인은 암페타민 과다 복용이라고 했다. 이에 1960년대 중반부터 각 경기 연맹은 약물 복용을 금지하기 시작했으며 1967년에는 IOC도 약물 복용 금지에 동참했다.  \n올림픽에서 약물 복용 양성 반응이 나와서 메달을 박탈당한 첫 번째 사례로는 1968년 하계 올림픽의 근대 5종 경기에 출전해 동메달을 딴 한스 군나르 리렌바르가 있다. 그는 경기 후 도핑검사 결과 알코올을 복용한 것으로 확인되어 메달을 박탈당했다. 도핑 양성 반응으로 메달을 박탈당한 것으로 가장 유명한 사람은 1988년 하계 올림픽 육상 100m 경기에서 금메달을 땄으나 도핑 검사 결과 스타노졸롤을 복용한 것으로 확인돼 금메달을 박탈당한 캐나다 선수인 벤 존슨이 있다. 이에 따라 금메달은 2위를 했던 칼 루이스가 대신 받았다.  \n1990년대 후반, 여러 뜻있는 사람들이 도핑과의 전쟁을 선포하면서 1999년에 세계반도핑기구(WADA)를 설립한다. 2000년 하계 올림픽과 2002년 동계 올림픽 때는 약물 양성 반응을 보인 선수들이 급격히 증가했고, 역도와 크로스컨트리에서는 몇몇 선수들이 도핑 테스트에 걸려서 실격되기도 했다. 2006년 동계 올림픽 때는 메달리스트 한 명이 양성반응을 보여 메달을 

In [ ]:
#############################################
# Retriever를 포함한 전체 chain 을 구성
#
# 쿼리 -> (retriever) -> 문서들/쿼리 -> (Prompt Template) -> prompt -> (llm) -> 응답
#############################################
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_template(
    template="""# Instruction:
당신은 정보제공을 목적으로하는 유능한 AI Assistant 입니다.
주어진 context의 내용을 기반으로 질문에 답변을 합니다.
Context에 질문에 대한 명확한 정보가 있는 경우 그것을 바탕으로 답변을 합니다.
Context에 질문에 대한 명확한 정보가 없는 경우 "정보가 부족해 답을 할 수없습니다." 라고 답합니다.
절대 추측이나 일반 상식을 바탕으로 답을 하거나 Context 없는 내용을 만들어서 답변해서는 안됩니다.

# Context:
{context}

# 질문:
{query}
"""
)

retriever = vector_store.as_retriever(search_kwargs={"k":5})
model = ChatOpenAI(model="gpt-5-mini")

def format_str(docs:list) -> str:
    """
    docs: list[Document] - vector store 검색한 내용에서 page_content만 추출해서 반환하는 Runnable
    """
    return '\n\n'.join(doc.page_content for doc in docs)

chain = {
    "context": retriever | format_str,
    "query":RunnablePassthrough()
} | prompt | model | StrOutputParser()

In [ ]:
query = "올림픽에서 약물과 관련된 스캔들이 뭐가 있어지?"
response = chain.invoke(query)

In [26]:
print(response)

다음은 제공된 문맥에 나온 올림픽 관련 약물(도핑) 스캔들 및 주요 사건들입니다.

- 1904년: 마라톤 우승자 토머스 J. 힉스(미국)가 코치로부터 스트리크닌과 브랜디를 받고 경주에 임함(운동선수들의 약물 복용 사례 초기 사례로 언급됨).  
- 1960년 로마 대회: 사이클 개인도로 경기 중 덴마크 선수 크누드 에네마르크 옌센이 자전거에서 떨어져 사망했으며, 검시관들은 원인을 암페타민 과다 복용으로 판정.  
- 1967년: 각 경기 연맹이 도핑 금지 시작한 가운데 IOC도 1967년에 도핑 금지에 동참.  
- 1968년 멕시코 시티 대회: 근대 5종 동메달리스트 한스 군나르 리렌바르(문맥상 표기)가 경기 후 도핑 검사에서 알코올 복용이 확인되어 메달 박탈 — 올림픽에서 약물 양성 반응으로 메달을 박탈당한 첫 사례로 언급.  
- 1988년 서울 대회: 육상 100m 금메달리스트 벤 존슨(캐나다)이 스타노졸롤 복용으로 양성 판정되어 금메달 박탈(가장 유명한 도핑 사건 중 하나).  
- 1999년: 도핑과의 전쟁을 위한 기관인 세계반도핑기구(WADA) 설립(도핑 문제 대응의 중요한 전환점).  
- 2000년·2002년 올림픽: 도핑 양성 반응을 보인 선수들이 급격히 증가(역도·크로스컨트리 등에서 실격 사례 발생).  
- 2006년 동계 올림픽: 메달리스트 한 명이 양성반응을 보여 메달 반납.  
- 2008년 베이징 올림픽: WADA가 3,667명에 대해 소변·혈액 검사 실시; 일부 선수들이 출전 금지 조치를 당했고, 올림픽 기간 중에는 단 3명만이 도핑 검사에 걸림.  
- IOC가 만든 약물 판정 체계가 인정받아 다른 경기 연맹에서도 벤치마킹됨(문맥에 언급된 제도적 변화).

문맥에 없는 추가 사례나 세부사항은 제공할 수 없습니다.
